In [ ]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install bertopic sentence-transformers umap-learn hdbscan

In [ ]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import json

In [ ]:
with open("/content/Query4Code.jsonl", "r") as f:
  for line in f:
    data = [json.loads(line) for line in f]
print(len(data))
# not sure what to do here
# should we use both code and query or only code
docs = [f"code:{item['code']} query:{item['query']}"for item in data]

237199


In [ ]:
model = BERTopic(
    embedding_model="all-MiniLM-L6-v2",   # Efficient embedding model for quick computation
    umap_model=UMAP(
        n_neighbors=10,                  # Lower neighbors for tighter clusters
        n_components=5,                  # Dimensionality reduction
        min_dist=0.1,                    # Slight separation between clusters
        metric='cosine'                  # Cosine similarity for text data
    ),
    hdbscan_model=HDBSCAN(
        min_cluster_size=60,             # Increase to reduce topic count
        min_samples=15,                   # Fewer samples to prevent fragmentation
        metric='euclidean',              # Works well with UMAP output
        cluster_selection_method='eom'   # Keeps distinct clusters
    ),                       # Target topic count, reduces smaller ones
    top_n_words=10                       # Top words per topic
)
topics, probs = model.fit_transform(docs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(topics)
print(model.get_topic_info())
uniq_topics = set(topics)
for topic in uniq_topics:
  print(f"topic:{topic}, name:{model.get_topic(topic)}")

[12, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 209, -1, -1, -1, 358, 358, 358, 358, -1, -1, 6, 37, -1, -1, 222, 39, -1, -1, -1, 39, -1, 39, 151, 39, 39, 39, -1, -1, 81, 81, 39, -1, -1, 2, 2, -1, 318, -1, 2, 244, -1, 358, -1, -1, -1, 39, -1, -1, -1, -1, 87, 87, 39, -1, 220, -1, 83, -1, 39, -1, 39, 39, 39, 39, 39, 39, 39, 39, 2, 81, 81, 48, 81, -1, -1, -1, 161, -1, 2, 83, 39, -1, -1, -1, -1, -1, 37, 358, -1, 83, 83, -1, -1, -1, -1, -1, 39, 40, -1, -1, -1, 40, 40, 40, 40, 219, -1, 33, 33, 46, 46, -1, -1, 236, -1, -1, -1, 95, -1, -1, -1, -1, 15, -1, 248, -1, 11, 216, -1, -1, -1, 191, 191, 191, -1, -1, -1, -1, -1, -1, 191, -1, -1, -1, -1, -1, 188, -1, -1, -1, -1, -1, 2, -1, -1, 148, 2, -1, -1, -1, -1, 26, -1, 12, -1, -1, 16, 16, 16, 16, -1, 89, 62, 89, -1, 131, -1, 60, 60, -1, 60, -1, -1, -1, 59, -1, 16, -1, -1, -1, -1, -1, -1, -1, 256, 256, 131, -1, 60, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 27, 33, 150, 61, -1, 8, -1, -1, 8, 22, -1, -1, -1, -1, -1, -1, -1, 82, -1, -1, -1, -1, 66,

In [ ]:
import pandas as pd

topic_info_df = model.get_topic_info()
topic_info_df.to_csv("bertopic_topic_info.csv", index=False)

In [ ]:
df = pd.DataFrame({
    "document": docs,
    "topic": topics,
    "probability": probs
})
df.to_csv("document_topic_mapping.csv", index=False)

In [ ]:
print(max(topics))

465


In [ ]:
import json
from collections import defaultdict

topic_data_map = defaultdict(list)

for i, topic_assignment in enumerate(topics):
    document_data = data[i]
    document_probs = probs[i]

    topic_data_map[topic_assignment].append((document_data, document_probs))


json_output = json.dumps(topic_data_map, indent=4)

with open("cluster.json", "w") as f:
    f.write(json_output)

print("Topic-document map saved to topic_document_map.json")

Topic-document map saved to topic_document_map.json
